In [ ]:
!pip install langdetect
!pip install contractions



[notice] A new release of pip available: 22.2.2 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     -------------------------------------- 981.5/981.5 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=960ababbc79211dd53b4b20dcf06d2e80aa0a110037bfb8a3b2357462b44a683
  Stored in directory: c:\users\namrata thakur\appdata\local\pip\cache\wheels\d1\c1\d9\7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [ ]:
import pandas as pd
import numpy as np
from langdetect import detect
import string
import matplotlib.pyplot as plt
from glob import glob
#text preprocess file
import unicodedata
import contractions         
import re
import nltk
from nltk import sent_tokenize
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
from sklearn.model_selection import train_test_split

nltk.download('averaged_perceptron_tagger')

In [ ]:
# get list of file names
#csv_files = glob('../data/ReviewData/*.csv')#
csv_files = glob('review_consolidated.csv')
# Get the default list of stopwords
stopwords_list = stopwords.words('english')
# Create a new list without negation words
new_stopwords_list = [word for word in stopwords_list if word not in ['not', 'never', 'no', 'none']]


In [ ]:
# validating count of columns for all files and fixing the troublesome files

df = []

for f in csv_files:
    try:
        d = pd.read_csv(f)
        if d.shape[1]!=9:
            print(f)
    except:
        d = pd.read_csv(f,encoding='ISO-8859-1')
        d = d.drop([c for c in d.columns if 'Unnamed' in c], axis=1)
        d = d.rename(columns={'Merlion_Park':'place'})
    
    df.append(d)



C:\Users\Namrata Thakur\AppData\Local\Temp\ipykernel_19672\2829852895.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(f)


In [ ]:
# concatenating all dataframes 
data = pd.concat(df, ignore_index=True)

# save data in to dataframe
# data.to_csv('../data/review_consolidated.csv', index=False)

# Load the data

In [ ]:
data = pd.read_csv('../data/review_consolidated_drive.csv')

/var/folders/wd/s9qh73dx1hbc5dsw2drwrtph0000gn/T/ipykernel_13530/2468390757.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/review_consolidated_drive.csv')


In [ ]:
data.head()

,place,reviewer,reviewer_location,reviewer_contributions,review_rating,review_type,review_date,review_title,review_text
0,River Wonders,jaytara,"Singapore, Singapore",648.0,3.0,NaN,"June 1, 2022",Poorly signposted,It's a long walk from the carpark to the River...
1,River Wonders,OceanMasterII,NaN,NaN,3.0,Family,"May 2, 2022",May 2022 Visit,Having visited Singapore Zoo and Night Safari ...
2,River Wonders,Oscardog16,"Adelaide, Australia",1014.0,4.0,NaN,"December 27, 2022",Nice size park to spend 3 hours,Many walkways are covered so you are not in th...
3,River Wonders,CSK,"Singapore, Singapore",114.0,2.0,NaN,"May 4, 2022",Nothing much. Tix only worth for the pandas.,The entrance fee for adults varies from $38-$4...
4,River Wonders,SmithySy57,"Woking, UK",8.0,1.0,Family,"January 3, 2020",Waste of a lot of time,Disappointing visit with very long queues mult...


# Preprocess Text

In [ ]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)


def remove_non_ascii(text):
    # Define a list of ASCII characters
    ascii_letters = string.ascii_letters + string.digits + string.punctuation + ' '
    
    # Filter out non-ASCII characters
    ascii_text = ''.join(filter(lambda x: x in ascii_letters, text))
    
    return ascii_text

def remove_special_characters(text):
    # Define the pattern to match special characters
    pattern = r'[^a-zA-Z\s]'
    # Remove the special characters
    filtered_text = re.sub(pattern, '', text)
    return filtered_text

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(text):
    # Define a list of stopwords for the English language
    stop_words = set(stopwords.words('english'))
    # Split the text into words
    words = text.split()
    # Remove the stopwords
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Join the filtered words back into a text string
    filtered_text = ' '.join(filtered_words)
    return filtered_text



In [ ]:
%%time
def detect_lang(reviews):
    cn = [] 
    for x in reviews:
        try:
            lang = detect(x)
        except:
            lang = 'others'
        cn.append(lang)
    return cn

data['language'] = detect_lang(data['review_text'].values)

CPU times: user 9min 25s, sys: 4.04 s, total: 9min 29s
Wall time: 9min 31s


In [ ]:
# remove non english text
data1 = data[data['language']=='en']
data1.to_csv('review_consolidated_lang.csv')

In [ ]:
wnl = nltk.WordNetLemmatizer()
def pos_tagger(review):
    #print(review)
    lemma_text = []
    text = nltk.word_tokenize(review)
    pos_text = nltk.pos_tag(text)
    for word, tag in pos_text:
        if tag[0] in ['N', 'V', 'J', 'R']:
            if tag in ('NN','NNPS','NNP','NNS','RB', 'RBR', 'RBS','VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ','ADV'):
                lemma_text.append(wnl.lemmatize(word,pos= tag[0].lower()))
            elif tag in ('JJR', 'JJS','JJ'):
                lemma_text.append(wnl.lemmatize(word, pos = 'a'))
            else:
                lemma_text.append(word)
    return lemma_text




In [ ]:
nltk.pos_tag(['bundling','interesting'])



[('bundling', 'VBG'), ('interesting', 'VBG')]

In [ ]:
def lemmatization_tagger(text):
    lemma_text = []
    #print(text)
    for review in tqdm(text):
#         print(review)
        clean_text = pos_tagger(review)
        lemma_text.append((' '.join(clean_text)))
    return lemma_text

In [ ]:
%%time
def preprocess(text):
    text = replace_contractions(text)
    text = remove_non_ascii(text)
    text = text.lower()
    text = remove_stopwords(text)
    text = remove_special_characters(text)
    return text


data1['preprocessed_text'] = data1['review_text'].apply(preprocess)

CPU times: user 24.5 s, sys: 1.94 s, total: 26.5 s
Wall time: 26.5 s


<timed exec>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
%%time
data1['lemmatized_clean_review_text'] = lemmatization_tagger(data1['preprocessed_text'])
data1.head()


100%|██████████████████████████████████| 195706/195706 [04:49<00:00, 676.70it/s]


CPU times: user 4min 45s, sys: 4.14 s, total: 4min 49s
Wall time: 4min 49s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,place,reviewer,reviewer_location,reviewer_contributions,review_rating,review_type,review_date,review_title,review_text,language,preprocessed_text,lemmatized_clean_review_text
0,River Wonders,jaytara,"Singapore, Singapore",648.0,3.0,NaN,"June 1, 2022",Poorly signposted,It's a long walk from the carpark to the River...,en,long walk carpark river wonders entrance insid...,long walk carpark river wonder entrance signpo...
1,River Wonders,OceanMasterII,NaN,NaN,3.0,Family,"May 2, 2022",May 2022 Visit,Having visited Singapore Zoo and Night Safari ...,en,visited singapore zoo night safari numerous ti...,visit singapore zoo night safari numerous time...
2,River Wonders,Oscardog16,"Adelaide, Australia",1014.0,4.0,NaN,"December 27, 2022",Nice size park to spend 3 hours,Many walkways are covered so you are not in th...,en,many walkways covered baking sun pm animal sh...,many walkway cover bake sun pm animal show boo...
3,River Wonders,CSK,"Singapore, Singapore",114.0,2.0,NaN,"May 4, 2022",Nothing much. Tix only worth for the pandas.,The entrance fee for adults varies from $38-$4...,en,entrance fee adults varies depending citizen ...,entrance fee adult varies depend citizen forei...
4,River Wonders,SmithySy57,"Woking, UK",8.0,1.0,Family,"January 3, 2020",Waste of a lot of time,Disappointing visit with very long queues mult...,en,disappointing visit long queues multiple times...,disappointing visit long queues multiple time ...


In [ ]:
# save data in to dataframe
data1.to_csv('../data/preprocessed_reviews_lemmatised.csv', index=False)


# Split the data in to train and test

In [ ]:
sampled, unseen = train_test_split(data1, random_state=0, test_size=0.5)
train, test = train_test_split(sampled, random_state=0, test_size=0.2)

In [ ]:
train.shape, test.shape, unseen.shape

((78282, 12), (19571, 12), (97853, 12))

Transforming the data using Tfidf Vectoriser and word2vec

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
import pickle

sentences = [sentence.split() for sentence in train['lemmatized_clean_review_text']]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)
w2v_words = w2v_model.wv.index_to_key

# Define the TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=10000)


In [ ]:

print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  10712
sample words  ['singapore', 'visit', 'place', 'see', 'go', 'garden', 'get', 'time', 'great', 'take', 'walk', 'well', 'view', 'good', 'day', 'zoo', 'also', 'night', 'show', 'food', 'many', 'beautiful', 'shop', 'enjoy', 'lot', 'animal', 'experience', 'really', 'area', 'bay', 'nice', 'worth', 'make', 'even', 'city', 'way', 'park', 'best', 'look', 'flower', 'light', 'hour', 'much', 'love', 'find', 'orchid', 'people', 'recommend', 'tour', 'different']


In [ ]:
x_train_tfidf = vectorizer.fit_transform(train['lemmatized_clean_review_text'])
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(vectorizer.get_feature_names(), list(vectorizer.idf_)))
     

/Users/yashnagogineni/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
def get_weighted_w2v_tfidf(vectorizer, sentences):
    # TF-IDF weighted Word2Vec

    tfidf_feat = vectorizer.get_feature_names() # tfidf words/col-names


    tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
    row=0
    for sent in tqdm(sentences): # for each review/sentence 
        sent_vec = np.zeros(100) # as word vectors are of zero length
        weight_sum =0; # num of words with a valid vector in the sentence/review
        for word in sent: # for each word in a review/sentence
            if word in w2v_words and word in tfidf_feat:
                vec = w2v_model.wv[word]
    #             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
                # to reduce the computation we are 
                # dictionary[word] = idf value of word in whole courpus
                # sent.count(word) = tf valeus of word in this review
                tf_idf = dictionary[word]*(sent.count(word)/len(sent))
                sent_vec += (vec * tf_idf)
                weight_sum += tf_idf
        if weight_sum != 0:
            sent_vec /= weight_sum
        tfidf_sent_vectors.append(sent_vec)
        row += 1
        
    print(len(tfidf_sent_vectors))
    return tfidf_sent_vectors

In [ ]:
train['vector'] = get_weighted_w2v_tfidf(vectorizer, sentences)

100%|████████████████████████████████████| 78282/78282 [02:49<00:00, 462.23it/s]


78282


In [ ]:
test_sentences = [sentence.split() for sentence in test['lemmatized_clean_review_text']]
unseen_sentences = [sentence.split() for sentence in unseen['lemmatized_clean_review_text']]

test['vector'] = get_weighted_w2v_tfidf(vectorizer, test_sentences)
unseen['vector'] = get_weighted_w2v_tfidf(vectorizer, unseen_sentences)

100%|████████████████████████████████████| 19571/19571 [00:42<00:00, 464.61it/s]
/Users/yashnagogineni/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


19571


100%|████████████████████████████████████| 97853/97853 [03:32<00:00, 459.66it/s]

97853


In [ ]:
train['is_train']='train'
test['is_train']='test'
unseen['is_train']='unseen'

to_sav = pd.concat([train, test, unseen], axis=0)
to_sav.to_csv('vector.csv',index=False)

In [ ]:
# def vectorize(sentence):
#     words = sentence.split()
#     words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
#     if len(words_vecs) == 0:
#         return np.zeros(100)
#     words_vecs = np.array(words_vecs)
#     return words_vecs

# x_train_tfidf = vectorizer.fit_transform(train['lemmatized_clean_review_text'])
# x_test_tfidf = vectorizer.transform(test['preprocessed_text'])
# x_unseen_tfidf = vectorizer.transform(unseen['preprocessed_text'])


# x_train_w2v = np.array([vectorize(sentence) for sentence in train['preprocessed_text']])
# x_test_w2v = np.array([vectorize(sentence) for sentence in test['preprocessed_text']])
# x_unseen_w2v = np.array([vectorize(sentence) for sentence in unseen['preprocessed_text']])


reference - https://spotintelligence.com/2023/02/15/word2vec-for-text-classification/